In [3]:
import math
import IPython
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider

def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ])

def rotate_quaternion(point, axis, angle: float) -> list:
    normalizedaxis = axis/np.linalg.norm(axis)
    q_inv = [math.cos(angle/2)]
    q = [math.cos(angle/2)]
    for value in normalizedaxis:
        q.append(math.sin(angle/2)*value)
        q_inv.append(-math.sin(angle/2)*value)
    pointquat = [0, point[0], point[1], point[2]]
    rotation = quaternion_multiply(quaternion_multiply(q, pointquat), q_inv)
    result = [float(i) for i in rotation[1:]]
    return(result)

print(rotate_quaternion([1, 0, 0], [0, 1, 0], np.pi/2))

[0.0, 0.0, -1.0000000000000002]


In [4]:
def plot_rotation_plotly(point, axis, angle, figure, label):
    t = np.linspace(0.0, angle, 100)
    x = np.zeros(100)
    y = np.zeros(100)
    z = np.zeros(100)
    for i in range(100):
        p = rotate_quaternion(point, axis, t[i])
        x[i] = p[0]
        y[i] = p[1]
        z[i] = p[2]
    figure.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines',
        name=label))
    figure.add_trace(go.Scatter3d(
        x=[0, axis[0]], y=[0, axis[1]], z=[0, axis[2]],
        mode='lines',
        name='Axis',
        # line=dict(color='pink', width=5)
        ))
    figure.add_trace(go.Scatter3d(
        x=[x[0]], y=[y[0]], z=[z[0]],
        mode='markers',
        name='Start',
        # line=dict(color='pink', width=5)
        ))
    figure.add_trace(go.Scatter3d(
        x=[x[-1]], y=[y[-1]], z=[z[-1]],
        mode='markers',
        name='End',
        # line=dict(color='pink', width=5)
        ))
    return rotate_quaternion(point, axis, angle)

def make3dplotly(name: str):
    fig = go.Figure()
    fig.update_layout(width=600, height=600, scene=dict(
        aspectmode = 'cube',
        xaxis=dict(nticks=4, range=[-1.2, 1.2]),
        yaxis=dict(nticks=4, range=[-1.2, 1.2]),
        zaxis=dict(nticks=4, range=[-1.2, 1.2]),
        bgcolor="white"),
        title=name, paper_bgcolor='lightgrey')

    fig.add_trace(go.Scatter3d(
        x=[-1.2, 1.2], y=[0, 0], z=[0, 0],
        mode='lines',
        name='x-axis',
        line=dict(color='black', width=5)
    ))

    fig.add_trace(go.Scatter3d(
        y=[-1.2, 1.2], x=[0, 0], z=[0, 0],
        mode='lines',
        name='y-axis',
        line=dict(color='black', width=5)
    ))

    fig.add_trace(go.Scatter3d(
        z=[-1.2, 1.2], y=[0, 0], x=[0, 0],
        mode='lines',
        name='z-axis',
        line=dict(color='black', width=5)
    ))
    return fig

In [5]:
p = (1.,0.,0.)
ax = (.05, .3, .5)
ang = 3.29

myfig = make3dplotly(
f'Rotation of ({p[0]}, {p[1]}, {p[2]}) by {ang} radians around \
({ax[0]}, {ax[1]}, {ax[2]})'
)
final_point = plot_rotation_plotly(p, ax, ang, myfig, 'rotation')
display(myfig)

In [6]:
def plot_rotation_mpl(point, axis, angle, figure, name):
    t = np.linspace(0.0, angle, 100)
    x = np.zeros(100)
    y = np.zeros(100)
    z = np.zeros(100)
    for i in range(100):
        p = rotate_quaternion(point, axis, t[i])
        x[i] = p[0]
        y[i] = p[1]
        z[i] = p[2]
    figure.plot(x, y, z, label=name)
    ax_x = np.array([0, axis[0]])
    ax_y = np.array([0, axis[1]])
    ax_z = np.array([0, axis[2]])
    figure.plot(ax_x, ax_y, ax_z, label='Axis')
    return(rotate_quaternion(point, axis, angle))

def make3dfigure():
    figure = plt.figure().add_subplot(projection='3d')
    figure.set_xlabel('x')
    figure.set_ylabel('y')
    figure.set_zlabel('z')
    return figure

def plot3axes(figure):
    figure.plot((-1.3,1.3), (0,0), (0,0), label='x')
    figure.plot((0,0), (-1.3,1.3), (0,0), label='y')
    figure.plot((0,0), (0,0), (-1.3,1.3), label='z')

def plot_widget_rotation(px, py, pz, axx, axy, axz, angle, elevation, azimuth):
    my_widget_fig = make3dfigure()
    my_widget_fig.view_init(elevation,azimuth)
    plot3axes(my_widget_fig)
    plot_rotation_mpl([px, py, pz], [axx, axy, axz], angle, my_widget_fig,
                      'path')
    plt.legend()
    plt.show()

def get_rotation_widgets():
    angle_slider = FloatSlider(
        value=ang, min=0., max=2*math.pi, step=0.1, description="Angle")
    axx_slider = FloatSlider(
        value=ax[0], min=0., max=1.0, step=0.1, description="Axis x")
    axy_slider = FloatSlider(
        value=ax[1], min=0., max=1.0, step=0.1, description="Axis y")
    axz_slider = FloatSlider(
        value=ax[2], min=0., max=1.0, step=0.1, description="Axis z")
    px_slider = FloatSlider(
        value=p[0], min=-1., max=1.0, step=0.1, description="Point x")
    py_slider = FloatSlider(
        value=p[1], min=-1., max=1.0, step=0.1, description="Point y")
    pz_slider = FloatSlider(
        value=p[2], min=-1., max=1.0, step=0.1, description="Point z")
    elevation_slider= IntSlider(
        value=35, min=-90, max=90,step=5, continuous_update=False)
    azimuth_slider= IntSlider(
        value=45, min=0, max=360,step=5, continuous_update=False)
    return [px_slider, py_slider, pz_slider, axx_slider, axy_slider,
            axz_slider, angle_slider, elevation_slider, azimuth_slider]

In [7]:
def normalize_quaternion(q):
    norm = np.linalg.norm(q)
    return q / norm

def quaternion_to_rotation_matrix(q):
    q = normalize_quaternion(q)
    w, x, y, z = q
    return np.array([
        [1 - 2*y**2 - 2*z**2, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x**2 - 2*y**2]
    ])

def rotate_vector(q, v):
    R = quaternion_to_rotation_matrix(q)
    return np.dot(R, v)

# Example usage:
q = np.array([1, 0, 0, 0])  # Identity quaternion
v = np.array([1, 0, 0])     # Vector along x-axis
rotated_v = rotate_vector(q, v)
print(rotated_v)

[1. 0. 0.]
